In [7]:
import json
with open ("test.json", "r") as f:
    lines = f.readlines()
print(type(lines))

<class 'list'>


In [51]:
import requests
import json
import random
import subprocess
import time
 
class GPT_new(object):
    def __init__(self, bns:str, address:str, model='gpt-4'):
        self.bns_name = bns
        self.headers = {'x_bd_timeout':'100000', 'Content-Type': 'application/json',}
        self.params = {'method': 'chatgptCompletions',}
        self.model = model
        self.address = address
 
    def get_bns_server(self):
        """
        Summary: 通过bns获取ip port
 
        Args:
            bns_name: bns
 
        Returns:
        """
        try:
            result = []
            cmd = "get_instance_by_service -a %s | awk '{if($5==0){print $2,$4}}'" % self.bns_name
            popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
            for line in popen.stdout.readlines():
                line = line.decode('utf-8')
                host, port = line.strip('\n').split()
                ip_ins= "{}:{}".format(host, port)
                result.append(ip_ins)
            return result
        except:
            return []
        
    def request(self, data):
        json_data = {
            'model': self.model,
            'app_code': 'wenku_strategy',
            'messages': data,
            'temperature': 0.7,
            'n': 1,
            # 'max_tokens': 4096,
            'presence_penalty': 0.5,
            'frequency_penalty': 0.5,
            'user': '123asd',
        }
        bns_list = self.get_bns_server()
        random_server = random.choice(bns_list)
 
        response = requests.post(url=f'http://{random_server+self.address}', params=self.params, headers=self.headers, data=json.dumps(json_data))
        return response
 
def get_gpt4_xiaoshuo_res(system:str, query:str, history:list=[], model:str='gpt-4'):
    gpt4 = GPT_new(bns='group.novel-author-online.novel.all', address='/novelinnerapi/aigc/completions/chat', model=model)
 
    input = [{'role':'system', 'content':'You are GPT-4, a large language model trained by OpenAI, based on the GPT-4 architecture.'}] #\nKnowledge cutoff: 2022-01\nCurrent date: 2023-10-27
    if len(system)>0:
        input[0]['content'] = system
 
    for role, content in history:
        input.append({"role": role, "content": content})
 
    input.append({"role": "user", "content": query})
 
    for _ in range(3):
        time.sleep(0.1)
        error = []
        response = gpt4.request(input)
        try:
            # res = response.json()['data']['choices'][0]['message']['content']
            res = response.json()['choices'][0]['message']['content']
            return res
        except:
            time.sleep(3)
            error.append(response.json())
            print(response.json())
            if 'content_filter' in response.text:
                return '-2'
            elif len(response.json())==0:
                print('response is null:', response.json())
    
    res = '-1'
    print('Error: ', res, '（调用三次失败）')
    return res

def get_gpt35_xiaoshuo_res(system:str, query:str, history:list=[]):
    gpt4 = GPT_new(bns = "group.novel-author-online.novel.all", address='/novelinnerapi/aigc/completions/gpt', model='gpt-35-turbo')

    input = [{'role':'system', 'content':'You are ChatGPT, a large language model trained by OpenAI, based on the ChatGPT architecture.'}] #\nKnowledge cutoff: 2022-01\nCurrent date: 2023-10-27
    if len(system)>0:
        input[0]['content'] = system

    for role, content in history:
        input.append({"role": role, "content": content})

    input.append({"role": "user", "content": query})

    for _ in range(3):
        # time.sleep(0.1)
        response = gpt4.request(input)
        try:
            res = response.json()['data']['choices'][0]['message']['content']
            # res = response['choices'][0]['message']['content']
            return res
        except:
            print(response.text)
            continue
    
    res = '-1'
    print('Error: ', res, '（调用三次失败）')
    return res



def get_gpt4_xiaoshuo_se_res(system:str='', query:str='',  history:list=[]):
    # if len(query)==0:
    #     return '输入query为空', []
    return get_gpt4_xiaoshuo_res(system=system, query=query, history=history, model='gtp-4')
 
def get_gpt4_xiaoshuo_ca_res(system:str='', query:str='', history:list=[]):
    # if len(query)==0:
    #     return '输入query为空', []
    return get_gpt4_xiaoshuo_res(system=system, query=query, history=history, model='gpt-4')    
 
# 15 qps
# result = get_gpt4_xiaoshuo_se_res(system='你是一个足球专家', query='我刚刚问你什么问题', history=[('user', '2018世界杯冠军是谁？'),('assistant','2018年世界杯冠军是**法国国家男子足球队')])
# print('gpt4 xiaoshuo se:', result)
 
# # 2 qps
# result = get_gpt4_xiaoshuo_ca_res(system='你是一个足球专家', query='我刚刚问你什么问题', history=[('user', '2018世界杯冠军是谁？'),('assistant','2018年世界杯冠军是**法国国家男子足球队')])
# print('gpt4 xiaoshuo ca:', result) 

# def get_gpt35_xiaoshuo_se_res(query, system:str='', history:list=[]):
#     if len(query)==0:
#         return '输入query为空', []
#     return get_gpt4_xiaoshuo_res(system=system, query=query, history=history, model='gtp-3.5')
 
# def get_gpt35_xiaoshuo_ca_res(query, system:str='', history:list=[]):
#     if len(query)==0:
#         return '输入query为空', []
#     return get_gpt4_xiaoshuo_res(system=system, query=query, history=history, model='gpt-3.5')    
 
        
def get_gpt4_tieba_res(system:str, query:str, history:list=[]):
    gpt4 = GPT_new(bns = "group.flow-tieba-go-aigc-processer.PANDORA.all", address='/service/aigc_processer')

    input = [{'role':'system', 'content':'You are GPT-4, a large language model trained by OpenAI, based on the GPT-4 architecture.'}] #\nKnowledge cutoff: 2022-01\nCurrent date: 2023-10-27
    if len(system)>0:
        input[0]['content'] = system

    for role, content in history:
        input.append({"role": role, "content": content})

    input.append({"role": "user", "content": query})

    for _ in range(3):
        time.sleep(5)
        response = gpt4.request(input)
        try:
            res = response.json()['data']['choices'][0]['message']['content']
            # res = response['choices'][0]['message']['content']
            return res
        except:
            print(response.json())
            continue
    
    res = '-1'
    print('Error: ', res, '（调用三次失败）')
    return res

def get_gpt35_tieba_res(system:str, query:str, history:list=[]):
    gpt35 = GPT_new(bns = "group.flow-tieba-go-aigc-processer.PANDORA.all", address='/service/aigc_processer', model='gpt-3.5')

    input = [{'role':'system', 'content':'You are ChatGPT, a large language model trained by OpenAI, based on the ChatGPT architecture.'}] #\nKnowledge cutoff: 2022-01\nCurrent date: 2023-10-27
    if len(system)>0:
        input[0]['content'] = system

    for role, content in history:
        input.append({"role": role, "content": content})

    input.append({"role": "user", "content": query})

    for _ in range(3):
        # time.sleep(0.1)
        response = gpt35.request(input)
        try:
            res = response.json()['data']['choices'][0]['message']['content']
            # res = response['choices'][0]['message']['content']
            return res
        except:
            print(response.json())
            continue
    
    res = '-1'
    print('Error: ', res, '（调用三次失败）')
    return res

    

In [44]:
import requests
import json
import os
import time
import random
import subprocess

from gpt_back import get_gpt35_tieba_res, get_gpt4_tieba_res, get_gpt35_xiaoshuo_res, get_gpt4_xiaoshuo_se_res


prompt = '''
You are a conversational dialogue generator. 

###Instructions###
Given a conversation context, your task is to generate five diverse positive response and five adversarial negative response respectively. 

###Positive Response### 
Positive response is valid for the conversation context. 

###Adversarial Negative Response###
Adversarial negative responses have a significant word overlap with the conversation context but are still irrelevant response, which may not have any relation to the context.
You need to choose some words (do not include stopwords such as "I", "you", "are", etc.) from the conversation context and use them directly or indirectly while writing the adversarial negative responses. 
Indirect usage here refers to using words closely related to the context words. 
For example,using synonyms,antonyms, homonyms, subwords, or other words that are known to frequently co-occur with the words in the context (e.g., the words  "flexibility" and "injuries" co-occur with "acrobatics").

The following are five examples of a conversation context and response, and the corresponding prediction. 

###Example###
Context: 
Is there something wrong?
I enjoy having your daughter in my class.
I'm glad to hear it.

Positive response: 
She is so brilliant.
Her behavior is good in the class.
I would love to hear that she knows every rules and regulation.
I was shocked to know that she is your daughter.
She answers all my questions.

Adversarial Negative Responses
I enjoy listening jazz music in my free time.
I need pin drop silence in the class. 
If I hear someone talking they will be sent out of the class.
I am glad you enjoyed the magic show organized by our team. 
I think there was something wrrong with the CCTV camera installed in the class.
This is the wrong method to solve the problem. Please be attentive in the class.
'''

data_type = 'test'
history = []
dataset = 'topicalchat'
with open (f"./data/{dataset}_{data_type}.json", "r") as f:
    dev_lines = json.load(f)
print(len(dev_lines))
model_name = 'gpt4'
start = 0
end = len(dev_lines)
fw = open(f'./outputs/{dataset}/{dataset}_{model_name}_both_{data_type}.json', 'a', encoding='utf-8')
system = ""

for id, line in enumerate(dev_lines[start:end]):
    print(id)
    # sample = json.loads(line)
    sample = line
    content = prompt + "\n" + "\n" + "Context: \n"
    pos_list = []
    adv_list = []
    context = ""
    for i, c in enumerate(sample["context"]):
        if i % 2 == 0:
            # ss = "FS: " + c + "\n"
            ss = c + "\n"
            context += ss
        else:
            # ss = "SS: " + c + "\n"
            ss = c + "\n"
            context += ss


    query = content + context + "\n" 
    # contents = context + "\n" + "Response: " + "\n" + n
    if model_name == 'gpt4':
        response = get_gpt35_xiaoshuo_se_res(system, query, history=history)
    else:
        response = get_gpt35_tieba_res(system, query, history=history)
    if response == '-1':
        print('调用失败')
    else:
        # print(context)
        pos_list.append(response.strip().strip('\n').strip('\n').split('\n')[1:6])
        # print(pos_list)
        adv_list.append(response.strip().strip('\n').strip('\n').split('\n')[8:13])
        tmp = {'id': id+start, 'context': context, 'positive': pos_list, 'adversarial': adv_list}
        fw.write(json.dumps(tmp, indent=4, separators=(',', ': '), ensure_ascii=False) + '\n')

In [45]:
import requests
import json
import os
import time
import random
import subprocess
from gpt_back import get_gpt35_tieba_res, get_gpt4_tieba_res, get_gpt35_xiaoshuo_res, get_gpt4_xiaoshuo_se_res

def main():


    prompt = '''
    You are a classifier. Given a conversation context, which includes 2 speakers[annotated as FS(FirstSpeaker) and SS(SecondSpeaker)], \
        and a response. Your task is to classify this response whether is positive or negative. \
    
    ###Positive Response### 
    Positive response is valid for the conversation context. 

    ###Adversarial Negative Response###
    Adversarial negative responses have a significant word overlap with the conversation context but are still irrelevant response, which may not have any relation to the context.
    You need to choose some words (do not include stopwords such as "I", "you", "are", etc.) from the conversation context and use them directly or indirectly while writing the adversarial negative responses. 
    Indirect usage here refers to using words closely related to the context words. 
    For example,using synonyms,antonyms, homonyms, subwords, or other words that are known to frequently co-occur with the words in the context (e.g., the words  "flexibility" and "injuries" co-occur with "acrobatics").

    Your output format is only the “Positive” or “Negative”.
    
    ###Example###
    The following are five examples of a conversation context and response, and the corresponding prediction. \

    Example 1:
    Context: 
    FS: Is there something wrong?
    SS: I enjoy having your daughter in my class.
    FS: I'm glad to hear it.

    Response: 
    She is so brilliant.
    Predicted： Positive

    Example 2:
    Context: 
    FS: Is there something wrong?
    SS: I enjoy having your daughter in my class.
    FS: I'm glad to hear it.

    Response: 
    I enjoy having your daughter in my class.
    Prediction: Positive

    Example 3:
    Context: 
    FS: Is there something wrong?
    SS: I enjoy having your daughter in my class.
    FS: I'm glad to hear it.

    Response: 
    I'm glad to hear it.
    Prediction: Positive

    Example 4:
    Context: 
    FS: We have to pick up Conrad before the party.
    SS: Alright, no problem.
    FS: We're supposed to meet him at Cal's Bar at 10

    Response: 
    I pushed the problem aside; at present it was insoluble.",
    Prediction: Negative

    Example 5:
    Context: 
    FS: Is there something wrong?
    SS: I enjoy having your daughter in my class.
    FS: I'm glad to hear it.

    Response: 
    I think there was something wrrong with the CCTV camera installed in the class.
    Prediction: Negative
    '''


    data_type = 'train'
    dataset = 'topicalchat'

    model_name = 'gpt4'
    
    system = ""

    input_path = f"./outputs/{dataset}/{dataset}_{model_name}_both.json"

    history = []
    with open (input_path, "r") as f:
        dev_lines = json.load(f)
        # f.readlines()
    start = 712
    end = len(dev_lines)
    fw = open(f'./outputs/{dataset}/{dataset}_{model_name}_class_{data_type}.json', 'a', encoding='utf-8')
    system = ""
    for id, line in enumerate(dev_lines[start:end]):
        # print(id)
        # sample = json.load(line)
        sample = line

        content = prompt + "\n" + "\n" + "Context: \n"

        context = ""
        # for i, c in enumerate(sample["context"]):
        for i, c in enumerate(sample["context"].split("\n")[:-1]):
            if i % 2 == 0:
                ss = "FS: " + c + "\n"
                context += ss
            else:
                ss = "SS: " + c + "\n"
                context += ss
        # print(context)
        # for n in sample["positive_responses"]:
        for n in sample["positive"][0]:
            query = content + context + "\n" + "Response: " + "\n" + n
            contents = context + "\n" 
            pos = "Response: " + "\n" + n
            # response = get_gpt4_tieba_res(system, query, history=history)
            response = get_gpt4_xiaoshuo_se_res(system, query, history=history)

            if response == '-1':
                print('调用失败')
            else:
                tmp = {'id': id+start, 'contents': contents, 'response' : pos, 'prediction': response, 'label': 'Positive'}
                fw.write(json.dumps(tmp, indent=4, separators=(',', ': '), ensure_ascii=False) + '\n')

        # for n in sample["adversarial_negative_responses"]:
        for n in sample["adversarial"][0]:
            query = content + context + "\n" + "Response: " + "\n" + n
            contents = context + "\n" + "Response: " + "\n" + n
            adv = "Response: " + "\n" + n
            # response = get_gpt4_tieba_res(system, query, history=history)
            response = get_gpt4_xiaoshuo_se_res(system, query, history=history)

            if response == '-1':
                print('调用失败')
            else:
                tmp = {'id': id+start, 'contents': contents, 'response' : adv, 'prediction': response, 'label': 'Negative'}
                fw.write(json.dumps(tmp, indent=4, separators=(',', ': '), ensure_ascii=False) + '\n')


if __name__=="__main__":
    main()